## Load Data for Retail Comments Embedding Demo ##

### Note - Be sure to populate bucket name and any required keys in vars.json ###
1. **demo_ofs.web_comment**.  Customer Comment History - OFS

In [1]:
import teradatasql, json

# load vars json
with open('../../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

bucket = session_vars['environment']['bucket']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    try:
        cur.execute('DROP TABLE demo_ofs.web_comment;')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    qry = '''
    CREATE MULTISET TABLE demo_ofs.web_comment
    
    (comment_id INTEGER,
    customer_id INTEGER,
    comment_text VARCHAR(50000) CHARACTER SET LATIN CASESPECIFIC,
    comment_summary VARCHAR(50000) CHARACTER SET LATIN CASESPECIFIC);
    '''
    cur.execute(qry)
    
    qry = f'''
    Insert into demo_ofs.web_comment
    SELECT comment_id, customer_id, comment_text, comment_summary
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/WEB_COMMENT/manifest.json'
                MANIFEST = 'TRUE'
                AUTHORIZATION = Repositories.PubAuth
            ) D;'''
    cur.execute(qry)